## Demo for SDK (latest)

This example shows how to use SDK to deploy a task. The notebook includes the following steps:
- [initialization](#initialization)
- [get `job_source_uri` (not covered in this demo)](#get-job_source_uri)
- [show available hardware info list](#show-available-hardware-information)
- [choose hardware config](#choose-hardware-config)
- [estimate payment amount](#estimate-payment-amount)
- [create task to get `task_uuid`](#create-task)
- [submit payment](#submit-payment)
- [validate payment](#validate-payment-to-deploy-task)
- [follow up task status](#follow-up-task-status-optional)
- [show result](#show-result)



### Initialization

- For test version, get `API_KEY` in dashboard page: https://orchestrator-test.swanchain.io
- For prod version, get `API_KEY` in dashboard page: https://orchestrator.swanchain.io

In [7]:
import sys
sys.path.insert(0, '..')

import os
import time
import dotenv
dotenv.load_dotenv("../.env")
from swan import SwanAPI , MCSAPI

dev_url = "https://swanhub-cali.swanchain.io"
# dev_url = "http://127.0.0.1:5008"

# Initialize the Swan Service
swan_api = SwanAPI(api_key=os.getenv("API_KEY"), environment=dev_url)
# for prod version
# swan_api = SwanAPI(api_key=os.getenv("API_KEY"))

# api_key = os.getenv("MCS_API_KEY")
# mcs_api = MCSAPI(api_key)


from swan.contract.swan_contract import SwanContract

contract = SwanContract(os.getenv('PK'), os.getenv('RPC'))

### Get `job_source_uri`
to simplify the process, here we use a existing `job_source_uri` which is a hello world application, used to create task.

In [8]:
# job_source_uri = 'https://test-api.lagrangedao.org/spaces/5117e998-c623-4837-8af9-2b7b0ce2de7f'
# job_source_uri = 'https://plutotest.acl.swanipfs.com/ipfs/QmT9PPa1fWLrGcmt7gBaoQxiKPnN5reo6bKYdMz6NWZRLM
# job_source_uri = 'https://plutotest.acl.swanipfs.com/ipfs/QmTRxhHrNjz1arPZaUzuAWSYHpZYciKRt39uzwsdRfsuxx'

# job_source_uri = 'https://plutotest.acl.swanipfs.com/ipfs/QmTKKtyegQqXqa9yJ5SymTe2ma6JeXvADPCw61eVoMncaV'

job_source_uri = 'https://plutotest.acl.swanipfs.com/ipfs/Qmcrvr8S86MwhmSTRJzLg4EdFvr1c22HFEbTdcsc7p1Y7E'

### Show available hardware information

In [9]:
hardwares = swan_api.get_hardware_config()
hardwares_info = [hardware.to_dict() for hardware in hardwares if hardware.status == "available"] 
hardwares_info

[{'id': 0,
  'name': 'C1ae.small',
  'description': 'CPU only · 2 vCPU · 2 GiB',
  'type': 'CPU',
  'reigion': ['North Carolina-US'],
  'price': '0.0',
  'status': 'available'},
 {'id': 1,
  'name': 'C1ae.medium',
  'description': 'CPU only · 4 vCPU · 4 GiB',
  'type': 'CPU',
  'reigion': ['North Carolina-US'],
  'price': '1.0',
  'status': 'available'},
 {'id': 12,
  'name': 'G1ae.small',
  'description': 'Nvidia 3080 · 4 vCPU · 8 GiB',
  'type': 'GPU',
  'reigion': ['North Carolina-US'],
  'price': '10.0',
  'status': 'available'},
 {'id': 13,
  'name': 'G1ae.medium',
  'description': 'Nvidia 3080 · 8 vCPU · 16 GiB',
  'type': 'GPU',
  'reigion': ['North Carolina-US'],
  'price': '11.0',
  'status': 'available'},
 {'id': 73,
  'name': 'R1ae.medium',
  'description': 'Nvidia 3080 · 8 vCPU · 32 GiB',
  'type': 'GPU',
  'reigion': ['North Carolina-US'],
  'price': '22.0',
  'status': 'available'}]

### Choose hardware config

choose the hardware with its config name or hardware id

in hardware config, `cfg_name` and `hardware_id` will be used in the steps to deploy task (create task and submit payment)

In [10]:
cfg_name = 'C1ae.medium' #"G1ae.medium"
hardware = [hardware for hardware in hardwares if hardware.name == cfg_name][0]
print(f"{hardware.name=}, {hardware.id=}, {hardware.region}")

hardware_id = hardware.id
print(f"The chosen {hardware_id=}")

hardware.name='C1ae.medium', hardware.id=1, ['North Carolina-US']
The chosen hardware_id=1


### Estimate Payment amount

In [11]:
duration_hour = 1 # hour
amount = contract.estimate_payment(hardware_id, duration_hour)
print(amount)

1000000000000000000


### Create task

This step uses above information to create task, and get `task_uuid`, which is useful in submit payment step.

In [12]:
import json
duration=3600*duration_hour

result = swan_api.create_task(
    cfg_name=cfg_name, 
    region='North Carolina-US', 
    start_in=5, 
    duration=duration, 
    job_source_uri=job_source_uri, 
    paid=contract._wei_to_swan(amount),
    wallet_address=os.getenv('WALLET'),
)
print(json.dumps(result, indent=2))
task_uuid = result['data']['task']['uuid']

ERROR:root:SwanAPIRequestException: No C1ae.medium machine in Quebec-CA.Traceback (most recent call last):
  File "..\swan\api\swan_api.py", line 158, in create_task
    raise SwanAPIException(f"No {cfg_name} machine in {region}.")
swan.common.exception.SwanAPIException: SwanAPIRequestException: No C1ae.medium machine in Quebec-CA.



null


TypeError: 'NoneType' object is not subscriptable

### Submit Payment

This step is using `task_uuid`, `hardware_id` and `duration` to submit payment via **ClientPayment** contract.

In [ ]:
tx_hash = contract.submit_payment(task_uuid, hardware_id, duration)
print(tx_hash)

0x4f18c5779e15cd29c7ec82faf02286f8fd1c854485f0e37898de93c424430c64


### Validate Payment to deploy task

This step will validate the payment and then make task eligible for assigning if validation successful

In [ ]:
if result_validation := swan_api.validate_payment(
    tx_hash=tx_hash,
    task_uuid=task_uuid
):
    print(json.dumps(result_validation, indent=2))
else:
    print('validation failed')

{'tx_hash': '0x4f18c5779e15cd29c7ec82faf02286f8fd1c854485f0e37898de93c424430c64', 'task_uuid': 'bf7d5ef9-6450-4b82-8ebe-1e27c6b1aa4c'}
{
  "data": {
    "task": {
      "created_at": "1713815905",
      "end_at": "1713819504",
      "leading_job_id": null,
      "refund_amount": null,
      "status": "created",
      "task_detail_cid": "https://data.mcs.lagrangedao.org/ipfs/QmYys2w3kUTAFRU7r9UjwUhaaUNRFwg7VUcAXhyyEqYqMR",
      "tx_hash": null,
      "updated_at": "1713815916",
      "uuid": "bf7d5ef9-6450-4b82-8ebe-1e27c6b1aa4c"
    }
  },
  "message": "Task payment validated successfully.",
  "status": "success"
}


### follow up task status (optional)
The following step is optional, shows information when waiting for task being deployed.

In [ ]:
# Check task info
info = swan_api.get_deployment_info(task_uuid=task_uuid)
print(json.dumps(info, indent=2))

{
  "data": {
    "computing_providers": [],
    "jobs": [],
    "task": {
      "created_at": "1713815905",
      "end_at": "1713819504",
      "leading_job_id": null,
      "refund_amount": null,
      "status": "created",
      "task_detail_cid": "https://data.mcs.lagrangedao.org/ipfs/QmYys2w3kUTAFRU7r9UjwUhaaUNRFwg7VUcAXhyyEqYqMR",
      "tx_hash": null,
      "updated_at": "1713815916",
      "uuid": "bf7d5ef9-6450-4b82-8ebe-1e27c6b1aa4c"
    }
  },
  "message": "fetch task info for task_uuid='bf7d5ef9-6450-4b82-8ebe-1e27c6b1aa4c' successfully",
  "status": "success"
}


### Show result

`job_real_uri` is for show the result of application you deployed.

In [13]:
r = swan_api.get_real_url(task_uuid)
print(r)

NameError: name 'task_uuid' is not defined

In [ ]:
import requests
import json

headers = {
    'Content-Type': 'application/json',
}

response = requests.get(r[0], headers=headers)

try:
    print(json.dumps(response.json(), indent=4))
except Exception as e:
    print(e)
    print(response)


IndexError: list index out of range